In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snb
import os,sys
from datetime import datetime

In [80]:
import keras
from keras.models import Sequential
from keras.layers import (
    LSTM, 
    Dropout, 
    Dense, 
    Activation
)
from tensorflow.keras.callbacks import (
    TensorBoard,
    EarlyStopping,
    ModelCheckpoint
)
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [17]:
input_file_name="seattle-weather.csv"
input_file_path=os.path.dirname(os.getcwd())
print(input_file_path)
input_file_path=os.path.join(input_file_path,"data")
print(input_file_path)
input_file_path=os.path.join(input_file_path,input_file_name)

print(input_file_path)
print("E:/GitHubFIles/py_play_ground/dl/simple_rnn/LSTM/data/seattle-weather.csv")

e:\GitHubFIles\py_play_ground\dl\simple_rnn\LSTM
e:\GitHubFIles\py_play_ground\dl\simple_rnn\LSTM\data
e:\GitHubFIles\py_play_ground\dl\simple_rnn\LSTM\data\seattle-weather.csv
E:/GitHubFIles/py_play_ground/dl/simple_rnn/LSTM/data/seattle-weather.csv


In [18]:
df=pd.read_csv(input_file_path)

In [19]:
df.shape

(1461, 6)

In [20]:
df.head()

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain


In [21]:
df.columns

Index(['date', 'precipitation', 'temp_max', 'temp_min', 'wind', 'weather'], dtype='object')

In [22]:
df.describe()

,precipitation,temp_max,temp_min,wind
count,1461.000000,1461.000000,1461.000000,1461.000000
mean,3.029432,16.439083,8.234771,3.241136
std,6.680194,7.349758,5.023004,1.437825
min,0.000000,-1.600000,-7.100000,0.400000
25%,0.000000,10.600000,4.400000,2.200000
50%,0.000000,15.600000,8.300000,3.000000
75%,2.800000,22.200000,12.200000,4.000000
max,55.900000,35.600000,18.300000,9.500000


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           1461 non-null   object 
 1   precipitation  1461 non-null   float64
 2   temp_max       1461 non-null   float64
 3   temp_min       1461 non-null   float64
 4   wind           1461 non-null   float64
 5   weather        1461 non-null   object 
dtypes: float64(4), object(2)
memory usage: 68.6+ KB


In [28]:
df.isnull().sum()

date             0
precipitation    0
temp_max         0
temp_min         0
wind             0
weather          0
dtype: int64

In [29]:
df.duplicated().sum()

np.int64(0)

In [34]:
type(df)

pandas.core.frame.DataFrame

In [37]:
training_set=df.iloc[:,2:3].values

In [38]:
training_set.shape

(1461, 1)

In [41]:
training_set[:10]

array([[12.8],
       [10.6],
       [11.7],
       [12.2],
       [ 8.9],
       [ 4.4],
       [ 7.2],
       [10. ],
       [ 9.4],
       [ 6.1]])

In [42]:
training_set.ndim

2

In [56]:
training_set[:1,0]

array([12.8])

In [44]:
len(df)

1461

In [59]:
def df_to_XY(training_set,window_size=10):
 X_train=[]
 y_train=[]

 for i in range(window_size,len(training_set)):
    X_train.append(training_set[i-window_size:i,0])
    y_train.append(training_set[i,0])
    
 X_train, y_train = np.array(X_train), np.array(y_train)
 return X_train, y_train

In [68]:
window_size = 10
x,y = df_to_XY(training_set,window_size)
print(len(x),len(y))
x_train = x[:800]
y_train = y[:800]
x_val = x[800:1000]
y_val = y[800:1000]
x_test = x[1000:]
y_test = y[1000:]

1451 1451


In [69]:
x.shape,y.shape

((1451, 10), (1451,))

In [70]:
x_train[:5,:],y_train[:5]

(array([[12.8, 10.6, 11.7, 12.2,  8.9,  4.4,  7.2, 10. ,  9.4,  6.1],
        [10.6, 11.7, 12.2,  8.9,  4.4,  7.2, 10. ,  9.4,  6.1,  6.1],
        [11.7, 12.2,  8.9,  4.4,  7.2, 10. ,  9.4,  6.1,  6.1,  6.1],
        [12.2,  8.9,  4.4,  7.2, 10. ,  9.4,  6.1,  6.1,  6.1,  5. ],
        [ 8.9,  4.4,  7.2, 10. ,  9.4,  6.1,  6.1,  6.1,  5. ,  4.4]]),
 array([6.1, 6.1, 5. , 4.4, 1.1]))

In [71]:
x_test.shape

(451, 10)

In [72]:
x_train=np.reshape(
    x_train,
    (x_train.shape[0],x_train.shape[1],1)
)
x_val=np.reshape(
    x_val,
    (x_val.shape[0],x_val.shape[1],1)
)
x_test=np.reshape(
    x_test,
    (x_test.shape[0],x_test.shape[1],1)
)


In [75]:
def construct_model(n, input_shape):
    model = Sequential()
    
    model.add(
        LSTM(
            units=n,
            return_sequences=True,
            input_shape=input_shape
        )
    )
    model.add(Dropout(0.2))
    model.add(Activation("relu"))
    
    model.add(
        LSTM(
            units=n,
            return_sequences=True
        )
    )
    model.add(Dropout(0.2))
    model.add(Activation("relu"))
    
    model.add(
        LSTM(
            units=n,
            return_sequences=True
        )
    )
    model.add(Dropout(0.2))
    model.add(Activation("relu"))
    
    model.add(
        LSTM(
            units=n,
            return_sequences=False
        )
    )
    model.add(Dropout(0.2))
    model.add(Activation("relu"))
    
    model.add(
        Dense(
            units=11
        )
    )
    model.add(Activation("relu"))
    
    return model


In [76]:
def compile_model(
        model,
        loss,
        optimizer,
        metrics
):
    model.compile(
        loss=loss,
        optimizer=optimizer,
        metrics=metrics
    )

    return model

In [81]:
def get_callbacks():
    file_name=datetime.now().strftime("%d_%m_%y__%Hh_%Mm_%Ss")
    main_dir=os.path.dirname(os.getcwd())
    file_path=os.path.join(main_dir,"data")
    os.makedirs(file_path,exist_ok=True)
    log_dir=os.path.join(file_path,file_name)

    tensorboard=TensorBoard(
        log_dir=log_dir,
        histogram_freq=1
    )

    '''
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=0,
        verbose=0,
        mode='auto',
        baseline=None,
        restore_best_weights=False,
        start_from_epoch=0
    )
    '''

    early_stopping=EarlyStopping(
        monitor="val_loss",
        patience=2,
        mode="auto",
        restore_best_weights=True,
        start_from_epoch=1
    )

    return [tensorboard,early_stopping]



def model_data(model):
    print(f"model summary : \n {model.summary()}")
    keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)



def train_model(
        model,
        x_train,
        y_train,
        batch_size,
        epochs,
        x_val,
        y_val,
        callbacks
):
    history=model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(
            x_val,
            y_val
        ),
        callbacks=get_callbacks()
    )

    return model,history



def training_history(history):
    print("Training history keys:", history.history.keys())
    for key, values in history.history.items():
        print(f"{key}: {values[-5:]}")  # Print last 5 values of each metric
    # print(history.head())



def evaluate_model(
        model,
        x_test,
        y_test
):
    loss,accuracy=model.evaluate(x_test,y_test)

    print(f"model loss : {loss}")
    print(f"model accuracy : {accuracy}")






In [84]:
n_units = 50
input_shape = (100, 1)  # Example shape for time-series

model = construct_model(n_units, input_shape)
model = compile_model(model, loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model_data(model)

# Assume x_train, y_train, x_val, y_val are already defined and preprocessed
callbacks = get_callbacks()
model, history = train_model(model, x_train, y_train, batch_size=32, epochs=20, x_val=x_val, y_val=y_val, callbacks=callbacks)

training_history(history)

# Evaluate on test set
evaluate_model(model, x_test, y_test)


c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 100, 50)        │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 100, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 100, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 100, 50)        │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 100, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 100, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 100, 50)        │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 100, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 100, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 11)             │           561 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 11)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 71,561 (279.54 KB)

 Trainable params: 71,561 (279.54 KB)

 Non-trainable params: 0 (0.00 B)

model summary : 
 None
You must install pydot (`pip install pydot`) for `plot_model` to work.
Epoch 1/20


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(32,), output.shape=(32, 11)